# Employee Group Project 23

##### By Tracy Gan, Aronn Laurel, Nikita Prabhu, Nolan Vu


### Data Description

The Employee dataset contains information about the employees in the company, including their personal information such as age, gender, educational background, years of experience, salary tier, and the city where they work. The dataset also contains their information in regards to the company, such as the year they joined, and if they were ever benched. Specifically, each variable is broken down into

| Variable Name     | Type of Variable | Description |
|-------------------|------------------|-------------|
| Education         | Categorical  | It represents the educational qualities of employees. There are three unique levels for this categorical variable, which are Bachelors, Masters, and PHD.   |
| JoiningYear       | Categorical  | Los Angeles  |
| City              | Categorical  | Chicago      |
| PaymentTier         | INSERT  | INSERT   |
| Age       | INSERT  | INSERT  |
| Gender              | INSERT  |INSERT  |
| EverBenched         | INSERT  | INSERT    |
| ExperienceInCurrentDomain       |  INSERT |INSERT  |
| LeaveOrNot              | INSERT  | INSERT     |

In total, there are 4653 rows of data for each column, where there are no missing values for any of the variables. This dataset also 